# Intro
体渲染直接在3D空间采样射线上的多个位置来计算射线颜色，不同于rasterization需要透视变换。

两者都约定相机坐标下的相机在原点，并且相机坐标轴由$\vec{\mathbf{x}},\vec{\mathbf{y}},\vec{\mathbf{z}}$三个轴定义，其中$\vec{\mathbf{z}}$与相机面向的方向相反，$\vec{\mathbf{y}}$为相机向上的方向，$\vec{\mathbf{x}}=-\vec{\mathbf{z}}\times \vec{\mathbf{y}}$（右手定则）。

体渲染的核心在于采样射线$\{\mathbf{p}(t)=\mathbf{o}+t\mathbf{v}|t\geq 0\}$上的点，射线的颜色为点颜色的加权和：
$$
C(\mathbf{o},\mathbf{v})=\int_0^{+\inf}w(t)c(\mathbf{p}(t),\mathbf{v})\mathrm{d}t,
$$
其中$w(t)$为权重函数。

在NeRF原文中，此权重函数$w(t)=T(t)\sigma(t)$。$T(t)=\exp(-\int_0^t\sigma(u)\mathrm{d}u)$

# 体渲染

<img src="../pasteImage/volume_rendering_world_camera.png">

渲染过程依然是涉及到旋转$\mathbf{R}_{w2c}$-投影$\mathbf{K}$两部分。这两部分最终可以由统一的线性变换$\mathbf{K}\mathbf{R}_{w2c}$表示。

### 旋转
旋转过程的目的是将世界坐标转化为相机坐标，$\mathbf{R}_{w2c}$的推导详见3D/viewport_tranformation.ipynb。如图（a），注意体渲染的相机坐标轴不同于光栅化渲染的相机坐标。

### 投影

给定3D点$\mathbf{P}(x,y,z,1)$，我们要获得其在focal length为$f$的image plane上的投影$\mathbf{p}$需要两次线性变化：
$$
\begin{equation}
\begin{aligned}
\mathbf{p}&=\mathbf{K}_f\Pi\mathbf{P}\\
\Pi&=
\begin{pmatrix}
1 &  &  & 0\\
& 1 & & 0\\
 &  & 1 & 0\\
\end{pmatrix}\\
\mathbf{K}_f&=\begin{pmatrix}
f &  & \\
& f & \\
 &  & 1 \\
\end{pmatrix}
\end{aligned}
\end{equation}
$$
而此时得到的$u,v$代表image plane coordinates，而我们最终希望获得非负的pixel idx $\bar{\mathbf{p}}=(\bar{u},\bar{v},1)$，其中$\bar{u},\bar{v}=Wu+c_u, Hv+c_v$，我们将其表示成线性变化：
$$
\begin{equation}
\begin{aligned}
\bar{\mathbf{p}}&=\mathbf{K}_s\mathbf{p}\\
\mathbf{K}_s&=
\begin{pmatrix}
W &  & c_u\\
& H & c_v\\
 &  & 1\\
\end{pmatrix}
\end{aligned}
\end{equation}
$$
而数据集中一般提供$\mathbf{K}=\mathbf{K}_s\mathbf{K}_f$，常被称作camera intrisic matrix，其表示形式如下：
$$
\begin{equation}
\begin{aligned}
\mathbf{K}&=
\begin{pmatrix}
f_x &  & c_u\\
& f_y & c_v\\
 &  & 1\\
\end{pmatrix}
\end{aligned}
\end{equation}
$$

# $K_f$推导与理解

### 先理解Projective Geometry

在2D世界中，一个点可以由如下其次坐标表示：
$$
\begin{equation}
\mathbf{p}=\begin{pmatrix}
x_1\\
x_2\\
x_3\\
\end{pmatrix}
\end{equation}
$$
它表示一个Euclidean space里一个2D的点：
$$
\begin{equation}
e(\mathbf{p})=\begin{pmatrix}
\frac{x_1}{x_3}\\
\frac{x_2}{x_3}\\
\end{pmatrix}
\end{equation}
$$
而如果我们固定$x_1,x_2$，任意调整$x_3$接近0的话，$e(\mathbf{p})$表示的点则会沿着向量$(x_1,x_2)^T$的方向不断远离原点。我们定义
$$
\begin{equation}
\mathbf{x}_c=\begin{pmatrix}
a\\
b\\
c\\
\end{pmatrix}\ \mathrm{where}\ (a,b)\neq (0,0)
\end{equation}
$$
为*projective plane* $\mathcal{P}^2$。类似地，我们可以定义
$$
\begin{equation}
\mathbf{x}_c=\begin{pmatrix}
a\\
b\\
d\\
c\\
\end{pmatrix}\ \mathrm{where}\ (a,b,d)\neq (0,0,0)
\end{equation}
$$
为*projective space* $\mathcal{P}^3$。

### 再理解$K_f$

因此*canonical projection matrix* $\Pi$实际上是将3D空间中的点$\mathbf{P}=(x,y,z,1)$转化成如下表示的*projective plane* $\mathcal{P}^2$：
$$
\begin{equation}
\mathbf{p}_z=\begin{pmatrix}
x\\
y\\
z\\
\end{pmatrix}
\end{equation}
$$
在image plane的focal length未知时，此时的$\mathbf{p}_z$表示的是一系列2D点的齐次坐标，而非一个3D点，因为$\Pi$是一个投影矩阵。如果给定focal length为$f$的情况下，我们有如下推导：
$$
\begin{equation}
\begin{aligned}
&\frac{u}{x}=\frac{v}{y}=\frac{f}{z}\\
\Rightarrow&\mathbf{p}=\begin{pmatrix}
f\frac{x}{z}\\
f\frac{y}{z}\\
1\\
\end{pmatrix}\\
\Rightarrow&\mathbf{K}_f=\begin{pmatrix}
f &  & \\
& f & \\
 &  & 1 \\
\end{pmatrix}
\end{aligned}
\end{equation}
$$

# Reference
https://courses.cs.duke.edu/cps274/fall15/notes/homogeneous-coordinates.pdf